# Welcome to the Modular Multimodal Data Fusion ML Pipeline for stress detection for the WESAD Database

## Table of contents






## Getting Started:
First, download necessary packages, if you are using a venv such as Conda, activate this first.

In [1]:
%pip install -r requirements.txt

## Data Installation
If you are on Linux, run this cell to download and extract the WESAD dataset automatically, otherwise download manually [here](https://uni-siegen.sciebo.de/s/HGdUkoNlW1Ub0Gx/download) and unzip the `WESAD` file into the `wesad` directory i.e. `wesad/WESAD/`

In [ ]:
%cd wesad && bash download_database.sh
%cd ..

## Data Preprocessing

This will automatically extract the biosensor data from the WESAD directory into several merged files in `.pkl` format.

This will likely take around 20 minutes depending on the machine.

In [14]:
from wesad.data_preprocessing.data_preprocessor import DataPreprocessor

preprocessor = DataPreprocessor()
# preprocessor.preprocess()

Now, using the `.pkl` files we will make it into a dataloader.

In [1]:
from wesad.data_preprocessing.dataloader import WesadDataset
from torch.utils.data import DataLoader

# Create the dataset
dataset = WesadDataset()

# Create DataLoader
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# Example usage in a training loop
for (train_data, test_data) in data_loader:
    merged, merged_y = train_data
    x_test, y_test = test_data
    
    print(merged, merged_y, x_test, y_test)
    break